### Load libraries

In [1]:
# Library for data visualisation
import pandas as pd
import numpy as np
from pathlib import Path
import pathlib
import os
from PIL import Image
import numpy as np
import json
from cv2 import cv2
import torch
from torchvision.ops import box_convert
from typing import List, Dict
from skimage.io import imread
import pickle

#for model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
import torch

import torchvision.transforms as T
from cv2 import cv2
import os
import json
import time
import pickle

import pathlib
from torch.utils.data import DataLoader
import sys
sys.path.append('../')

from tqdm import tqdm

#for model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
import torch


## Create a test train folder

In [4]:
import os
import random
import numpy as np
import shutil
from cv2 import cv2
import json
import pathlib
import sys
sys.path.append('../')

#Set path to root dir (wrist folder)
root_dir = '../literature/Other/supervisely/'

# Path to annotation dir
ann_dir = os.listdir(root_dir + 'wrist/ann')

# Path to image dir
img_dir = os.listdir(root_dir + 'wrist/img')



print('Number of annotations: {}'.format(len(ann_dir)))
print('Number of images: {}'.format(len(img_dir)))




Number of annotations: 20327
Number of images: 20327


In [5]:
#os.chdir('../')

print("The working directory of this project is:", os.getcwd())

The working directory of this project is: c:\Users\spiro\OneDrive\Υπολογιστής\Thesis\Project\master_thesis_dtu\notebooks


## Save images as RPG

In [6]:
os.mkdir("../literature/Other/supervisely/wrist/rpg_images")

In [7]:
from tqdm import tqdm
root_dir = pathlib.Path('../literature/Other/supervisely/wrist/img/')


for idx in tqdm(range(len(img_dir))):
    img_path = os.path.join(root_dir,img_dir[idx])
    ann_path = os.path.join('../literature/Other/supervisely/wrist/ann/',ann_dir[idx])


    img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # Convert the grayscale image to RGB and duplicate the grayscale channel
    img = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB) 
    cv2.imwrite("../literature/Other/supervisely/wrist/rpg_images/" +img_dir[idx] , img)


100%|██████████| 20327/20327 [20:46<00:00, 16.31it/s] 


## Create the pickles


In [8]:
from tqdm import tqdm
def data_fix(img_list,ann_list):

    root_dir = pathlib.Path('../literature/Other/supervisely')

    for idx in tqdm(range(len(img_list))):
        img_path = os.path.join(img_list[idx])
        ann_path = os.path.join(ann_list[idx])
        
        #target = []
        d = {}

        box = []
        boxes = []
        labels = []
        area = []

        classes = {'fracture' : 1, 'text' : 2}


        name = pathlib.PurePath(ann_path).name.split('.')[0]

        

        with open(ann_path) as json_file:
            
            #Load the JSON file
            data = json.load(json_file)


        for object_dict in data['objects']:
    
            # Check if object contains any fractures

            if object_dict['classTitle'] == "text" or object_dict['classTitle'] == "fracture":

                # Get points and convert them to int for display purposes
                top_left_point, bottom_right_point = object_dict['points']['exterior']
                top_left_point = list(map(int,top_left_point))
                bottom_right_point = list(map(int, bottom_right_point))
                box = (top_left_point+bottom_right_point)
                area.append((box[3] - box[1]) * (box[2] - box[0]))
                label = classes[object_dict['classTitle']]
                labels.append(label)
                boxes.append(box)

                # Check if the list is empty
                if len(box) == 0:
                    print("The list is empty!")
                

        boxes = torch.FloatTensor(boxes)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = torch.as_tensor(area, dtype=torch.int64)

        image_id = torch.tensor([idx])
                
        
        d["boxes"] = boxes
        d["labels"] = labels
        d["image_id"] = image_id
        d["area"] = area
        

        target = d

        img = img_path

        ready_data = {'image' : img,'target' : target}


        path = os.path.join(root_dir,'wrist','train_pickles', name + '.pickle')

        with open(path, 'wb') as handle:
            pickle.dump(ready_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
#Set path to root dir (wrist folder)
root_dir = pathlib.Path('../literature/Other/supervisely')

# Path to annotation dir
ann_dir = pathlib.Path(root_dir, 'wrist/ann')

# Path to image dir
img_dir = pathlib.Path(root_dir, 'wrist/rpg_images')

# Lists
list_files = lambda start_dir: [str(item) for item in start_dir.iterdir()]
ann_list = list_files(ann_dir)
img_list = list_files(img_dir)

print('Number of annotations: {}'.format(len(ann_list)))
print('Number of images: {}'.format(len(img_list)))

Number of annotations: 20327
Number of images: 20327


In [7]:
data_fix(img_list,ann_list)

100%|██████████| 20327/20327 [00:07<00:00, 2873.61it/s]


## Create train-test folders 


In [26]:
os.mkdir("../literature/Other/supervisely/wrist/validation_pickles")
os.mkdir("../literature/Other/supervisely/wrist/test_pickles")

In [37]:
#MOVE THE TEST PICKLES TO ANOTHER FOLDER
#Set path to root dir (wrist folder)
root_dir = pathlib.Path('../literature/Other/supervisely')
pickles = pathlib.Path(root_dir, 'wrist/train_pickles')
pickle_list = os.listdir(pickles)


random.seed(1234)
test_images = random.sample(pickle_list, 3000)


for pickle in test_images:
    shutil.move('../literature/Other/supervisely/wrist/train_pickles/'+pickle, '../literature/Other/supervisely/wrist/test_pickles/'+pickle)

In [39]:
#MOVE SOME IMAGES FOR VALIDATION
root_dir = pathlib.Path('../literature/Other/supervisely')
pickles = pathlib.Path(root_dir, 'wrist/train_pickles')
pickle_list = os.listdir(pickles)
validation_images = random.sample(pickle_list, 1000)

for pickle in validation_images:
    shutil.move('../literature/Other/supervisely/wrist/train_pickles/'+pickle, '../literature/Other/supervisely/wrist/validation_pickles/'+pickle)

In [41]:
print(len(os.listdir('../literature/Other/supervisely/wrist/train_pickles')))
print(len(os.listdir('../literature/Other/supervisely/wrist/validation_pickles')))
print(len(os.listdir('../literature/Other/supervisely/wrist/test_pickles')))

16318
1009
3000


In [6]:
import os

path = '../literature/Other/supervisely/wrist/rpg_images/1590_0984643851_01_WRI-R1_M015.png'
start_dir = '../'
relative_path = os.path.relpath(path, start_dir)

print(relative_path)

literature\Other\supervisely\wrist\rpg_images\1590_0984643851_01_WRI-R1_M015.png


In [12]:
class XRayDataSet(torch.utils.data.Dataset):
    

     def __init__(self, root):
        
        self.root = root
        self.instances = list(sorted(os.listdir(root)))
        

    
     def __len__(self):
        return len(self.instances) 

     def __getitem__(self,idx):
        
        instance = os.path.join(self.root, self.instances[idx])
        

        with open(instance, 'rb') as handle:
            data = pickle.load(handle)

        img_path = data['image']
        #for linux to work
        #start_dir = '../'
        #img_path = os.path.relpath(img_path, start_dir)
        #

        img = cv2.imread(img_path)
        img = T.ToTensor()(img).float()
        target = data['target']


        return img, target

     def read_images(inp, tar):
        return imread(inp), torch.load(tar)

In [59]:
#load train data
sys.path.append('../')

from src.data.my_rpg_dataset import collate_fn
train_dataset = XRayDataSet(pathlib.Path('../literature/Other/supervisely/wrist/validation_pickles'))
training_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=False, num_workers=0,collate_fn=collate_fn)

In [60]:
i=0
for imgs, annotations in tqdm(training_dataloader):
     
    imgs =list(img.to("cpu") for img in imgs)
    annotations = [{k: v.to("cpu") for k, v in t.items()} for t in annotations]

    # Check if the list is empty
    for annotation in annotations:

        if annotation['boxes'].numel() == 0:
            continue
    
print(i)
        


100%|██████████| 505/505 [00:37<00:00, 13.58it/s]

0


In [41]:
instances = os.listdir('../literature/Other/supervisely/wrist/train_pickles')

In [42]:
x[3827]

'1413_0931733132_01_WRI-R2_M010.pickle'

In [49]:
instance = os.path.join('../literature/Other/supervisely/wrist','train_pickles', instances[3823])
instance

'../literature/Other/supervisely/wrist\\train_pickles\\1411_0517013711_01_WRI-L1_F017.pickle'

In [50]:
with open(instance, 'rb') as handle:
    data = pickle.load(handle)

In [51]:
data

{'image': '..\\literature\\Other\\supervisely\\wrist\\rpg_images\\1411_0517013711_01_WRI-L1_F017.png',
 'target': {'boxes': tensor([]),
  'labels': tensor([], dtype=torch.int64),
  'image_id': tensor([4722]),
  'area': tensor([], dtype=torch.int64)}}